In [1]:
# Importing packages
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from fuzzywuzzy import process

C:\Users\troen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#datasets = ["acled1.csv", "acled2.csv", "acled3.csv", "acled4.csv", "acled5.csv"]
#dataframes = []

# Reading and processing each dataset
#for dataset in datasets:
 #   df = pd.read_csv("Data/" + dataset)

    # Columns to drop
  #  columns_to_drop = ['notes', 'source', 'source_scale', 'geo_precision', 
                       'latitude', 'longitude', 'tags', 'event_date', 
                       'civilian_targeting', 'location', 'region', 'timestamp',
                       'disorder_type', 'sub_event_type', 'admin1', 'admin2',
                       'admin3', 'actor1', 'assoc_actor_1', 'time_precision', 
                       'inter1', 'interaction', 'fatalities']

   # df = df.drop(columns=columns_to_drop)

    # Dropping duplicates in column event_id_cnty (because only actor change)
    #df = df.drop_duplicates(subset=['event_id_cnty'], keep='first')

    # Counting the number of occurrences of each event_type per country per year
    #pivot_table = df.pivot_table(index=['year', 'iso', 'country'], columns='event_type', aggfunc='size', fill_value=0)
    #pivot_table.reset_index(inplace=True)

    #dataframes.append(pivot_table)

# Merging all datasets together
#acled_df = pd.concat(dataframes, ignore_index=True)

# Fill NaN values with 0
# acled_df = acled_df.fillna(0)

In [2]:
# Importing data
main_df = pd.read_excel("Data/organizedviolencecy_v23_2.xlsx")
# armed_conflict_df = pd.read_csv("Data/armed_conflict_19462022.csv")

# mil_spending_df = pd.read_csv("Data/mil_spending.csv", skiprows=4)



# List of country codes to keep (recognized country)
recognized_country_codes = [
    "AFG", "ALA", "ALB", "DZA", "ASM", "AND", "AGO", "AIA", "ATA", "ATG", "ARG", "ARM", "ABW", "AUS", "AUT", "AZE",
    "BHS", "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BMU", "BTN", "BOL", "BES", "BIH", "BWA", "BVT", "BRA",
    "IOT", "BRN", "BGR", "BFA", "BDI", "CPV", "KHM", "CMR", "CAN", "CYM", "CAF", "TCD", "CHL", "CHN", "CXR", "CCK",
    "COL", "COM", "COG", "COD", "COK", "CRI", "CIV", "HRV", "CUB", "CUW", "CYP", "CZE", "DNK", "DJI", "DMA", "DOM",
    "ECU", "EGY", "SLV", "GNQ", "ERI", "EST", "SWZ", "ETH", "FLK", "FRO", "FJI", "FIN", "FRA", "GUF", "PYF", "ATF",
    "GAB", "GMB", "GEO", "DEU", "GHA", "GIB", "GRC", "GRL", "GRD", "GLP", "GUM", "GTM", "GGY", "GIN", "GNB", "GUY",
    "HTI", "HMD", "VAT", "HND", "HKG", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL", "IMN", "ISR", "ITA", "JAM",
    "JPN", "JEY", "JOR", "KAZ", "KEN", "KIR", "PRK", "KOR", "KWT", "KGZ", "LAO", "LVA", "LBN", "LSO", "LBR", "LBY",
    "LIE", "LTU", "LUX", "MAC", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT", "MHL", "MTQ", "MRT", "MUS", "MYT", "MEX",
    "FSM", "MDA", "MCO", "MNG", "MNE", "MSR", "MAR", "MOZ", "MMR", "NAM", "NRU", "NPL", "NLD", "NCL", "NZL", "NIC",
    "NER", "NGA", "NIU", "NFK", "MKD", "MNP", "NOR", "OMN", "PAK", "PLW", "PSE", "PAN", "PNG", "PRY", "PER", "PHL",
    "PCN", "POL", "PRT", "PRI", "QAT", "REU", "ROU", "RUS", "RWA", "BLM", "SHN", "KNA", "LCA", "MAF", "SPM", "VCT",
    "WSM", "SMR", "STP", "SAU", "SEN", "SRB", "SYC", "SLE", "SGP", "SXM", "SVK", "SVN", "SLB", "SOM", "ZAF", "SGS",
    "SSD", "ESP", "LKA", "SDN", "SUR", "SJM", "SWE", "CHE", "SYR", "TWN", "TJK", "TZA", "THA", "TLS", "TGO", "TKL",
    "TON", "TTO", "TUN", "TUR", "TKM", "TCA", "TUV", "UGA", "UKR", "ARE", "GBR", "USA", "UMI", "URY", "UZB", "VUT",
    "VEN", "VNM", "VGB", "VIR", "WLF", "ESH", "YEM", "ZMB", "ZWE"
]
# Count how many countries are in the recognized_country_codes list
print(len(recognized_country_codes))


249


In [3]:
import pycountry

# Create dictionaries to map country names to ISO codes and vice versa
country_name_to_iso = {country.name: country.alpha_3 for country in pycountry.countries}
iso_to_country_name = {country.alpha_3: country.name for country in pycountry.countries}

# Create DataFrames from the dictionaries
df_country_name_to_iso = pd.DataFrame(list(country_name_to_iso.items()), columns=['Country Name', 'ISO_Code'])
df_iso_to_country_name = pd.DataFrame(list(iso_to_country_name.items()), columns=['ISO_Code', 'Country Name'])

# Merge the DataFrames into a single DataFrame without duplicating "Country Name"
iso_df = pd.merge(df_country_name_to_iso, df_iso_to_country_name, on='ISO_Code', how='inner', suffixes=('_country', '_iso'))

# Rename the "Country Name" column to "Name"
iso_df.rename(columns={'Country Name_country': 'Name', 'ISO_Code': 'stateid'}, inplace=True)

# Drop the duplicate "Country Name_iso" column
iso_df.drop(columns=['Country Name_iso'], inplace=True)

# Only keep the countries in the recognized_country_codes list
iso_df = iso_df[iso_df['stateid'].isin(recognized_country_codes)]



In [ ]:

# Function to find closest match
def find_closest_match(country, country_list):
    closest_match = process.extractOne(country, country_list)
    return closest_match

# Initialize an empty mapping dictionary
mapping_dict = {}

# Iterate over rows in iso_df
for index, row in iso_df.iterrows():
    country_name_iso = row['Name']
    # Find the closest match in gleditsch_code DataFrame
    closest_match = find_closest_match(country_name_iso, gleditsch_code['countryname'])
    # Check if the score is above a certain threshold (adjust as needed)
    if closest_match[1] >= 90:
        # Retrieve the corresponding g_code
        g_code = gleditsch_code.loc[gleditsch_code['countryname'] == closest_match[0], 'g_code'].iloc[0]
        # Add the mapping to the dictionary
        mapping_dict[country_name_iso] = g_code

# Map the g_code values to iso_df based on the mapping dictionary
iso_df['g_code'] = iso_df['Name'].map(mapping_dict)


In [5]:
# Cleaning of GDP dataset: GDP between 1989 and 2022 in current US dollars
gdp_df = pd.read_csv("Data/gdp_world.csv", skiprows=4)

# Only keep Country Code is in recognized_country_codes
gdp_df = gdp_df[gdp_df["Country Code"].isin(recognized_country_codes)]

# Only keep relevant columns
gdp_df = gdp_df[["Country Name", "Country Code", "1989", "1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997",
                 "1998", "1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010",
                 "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]]

# Only keep the countries recognized in the recognized_country_codes list
gdp_df = gdp_df[gdp_df["Country Code"].isin(recognized_country_codes)]

# Backfill NaN values
gdp_df = gdp_df.fillna(method='bfill', axis=1)

# rename Country Code to stateid
gdp_df = gdp_df.rename(columns={"Country Code": "stateid"})

# Frontfill NaN values -> some countries miss their 2022 value
gdp_df = gdp_df.fillna(method='ffill', axis=1)


# Initialize the mapping dictionary
name_mapping = {
    "Congo, Dem. Rep.": "Congo",
    "Tanzania": "Tanzania, United Republic of",
    "Turkiye": "Türkiye",
    "Moldova": "Moldova, Republic of",
    "Bahamas, The": "Bahamas",
    "Macao SAR, China": "China",
    "Korea, Rep.": "Korea, Republic of",
    "Curacao": "Curaçao",
    "Egypt, Arab Rep.": "Egypt",
    "Congo, Rep.": "Congo",
    "Bolivia": "Bolivia, Plurinational State of",
    "Virgin Islands (U.S.)": "Virgin Islands, U.S.",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Hong Kong SAR, China": "Hong Kong",
    "British Virgin Islands": "Virgin Islands, British",
    "Yemen, Rep.": "Yemen",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "Gambia, The": "Gambia",
    # Add mappings for the countries not found in the ISO dataset
    "Micronesia, Fed. Sts.": "",  # Change this to the correct ISO name
    "Lao PDR": "",  # Change this to the correct ISO name
    "Iran, Islamic Rep.": "",  # Change this to the correct ISO name
    "Korea, Dem. People's Rep.": "",  # Change this to the correct ISO name
    "Kyrgyz Republic": "",  # Change this to the correct ISO name
    "St. Lucia": "",  # Change this to the correct ISO name
    "Slovak Republic": "",  # Change this to the correct ISO name
}

# Manually update the mapping dictionary with the correct ISO names
name_mapping["Micronesia, Fed. Sts."] = "Micronesia, Federated States of"
name_mapping["Lao PDR"] = "Lao People's Democratic Republic"
name_mapping["Iran, Islamic Rep."] = "Iran, Islamic Republic of"
name_mapping["Korea, Dem. People's Rep."] = "Korea, Democratic People's Republic of"
name_mapping["Kyrgyz Republic"] = "Kyrgyzstan"
name_mapping["St. Lucia"] = "Saint Lucia"
name_mapping["Slovak Republic"] = "Slovakia"

# Apply name mapping to GDP dataset
gdp_df['Country Name'] = gdp_df['Country Name'].map(name_mapping).fillna(gdp_df['Country Name'])

gdp_countries = gdp_df['Country Name'].unique()
iso_countries = iso_df['Name'].unique()

# Check if the same stateid is used for each country in both datasets
consistent_stateids = True
inconsistent_countries = []

for country in gdp_countries:
    if country in iso_countries:
        gdp_stateid = gdp_df.loc[gdp_df['Country Name'] == country, 'stateid'].iloc[0]
        iso_stateid = iso_df.loc[iso_df['Name'] == country, 'stateid'].iloc[0]
        
        if gdp_stateid != iso_stateid:
            consistent_stateids = False
            inconsistent_countries.append(country)

if consistent_stateids:
    print("The same stateid is used for each country in both datasets.")
else:
    print("The stateids are not consistent for the following countries:")
    print(inconsistent_countries)

# No data available for North Korea; estimation from tradingeconomics.com + linear interpolation

nk_gdp = {
    1989: 15770000000,
    1990: None,
    1991: None,
    1992: 11000000000,
    1993: None,
    1994: None,
    1995: 4850000000,
    1996: None,
    1997: None,
    1998: 11000000000,
    1999: None,
    2000: None,
    2001: None,
    2002: None,
    2003: None,
    2004: None,
    2005: None,
    2006: None,
    2007: None,
    2008: None,
    2009: None,
    2010: None,
    2011: None,
    2012: None,
    2013: None,
    2014: None,
    2015: None,
    2016: None,
    2017: None,
    2018: None,
    2019: None,
    2020: None,
    2021: None,
    2022: 20000000000
}

# Convert the dictionary to a DataFrame
nk_gdp_df = pd.DataFrame.from_dict(nk_gdp, orient='index', columns=['GDP'])

# Convert index to a column
nk_gdp_df.reset_index(inplace=True)
nk_gdp_df.rename(columns={'index': 'Year'}, inplace=True)

# Interpolate missing values linearly
nk_gdp_df['GDP'] = nk_gdp_df['GDP'].interpolate()

# Provided data
nk_gdp_values = {
    1989: 1.577000e+10, 1990: 1.418000e+10, 1991: 1.259000e+10, 1992: 1.100000e+10, 1993: 8.950000e+09,
    1994: 6.900000e+09, 1995: 4.850000e+09, 1996: 6.900000e+09, 1997: 8.950000e+09, 1998: 1.100000e+10,
    1999: 1.137500e+10, 2000: 1.175000e+10, 2001: 1.212500e+10, 2002: 1.250000e+10, 2003: 1.287500e+10,
    2004: 1.325000e+10, 2005: 1.362500e+10, 2006: 1.400000e+10, 2007: 1.437500e+10, 2008: 1.475000e+10,
    2009: 1.512500e+10, 2010: 1.550000e+10, 2011: 1.587500e+10, 2012: 1.625000e+10, 2013: 1.662500e+10,
    2014: 1.700000e+10, 2015: 1.737500e+10, 2016: 1.775000e+10, 2017: 1.812500e+10, 2018: 1.850000e+10,
    2019: 1.887500e+10, 2020: 1.925000e+10, 2021: 1.962500e+10, 2022: 2.000000e+10
}

# Locate the row with stateid equal to "PRK" in gdp_df
prk_row_index = gdp_df.index[gdp_df['stateid'] == 'PRK'].tolist()[0]

# Replace the values in columns 1989 to 2022 with the provided values
for year, gdp_value in nk_gdp_values.items():
    gdp_df.at[prk_row_index, str(year)] = gdp_value


# No data for Virgin Islands and Gibraltar: drop the columns (not used in main_df)
gdp_df = gdp_df[~gdp_df['stateid'].isin(['VGB', 'GIB'])]



The stateids are not consistent for the following countries:
['Congo']


C:\Users\troen\AppData\Local\Temp\ipykernel_20332\2016274278.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gdp_df = gdp_df.fillna(method='bfill', axis=1)
C:\Users\troen\AppData\Local\Temp\ipykernel_20332\2016274278.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gdp_df = gdp_df.fillna(method='ffill', axis=1)


In [6]:
# Cleaning the main_df

# Remove columns with disagregated deaths. Only keep "prior" information and cumulative total deaths.
main_df = main_df.loc[:, ['country_id_cy', 'country_cy', 'year_cy', 'region_cy', 'sb_dyad_count_cy', 'sb_dyad_ids_cy',
                          'sb_intrastate_dyad_count_cy', 'sb_intrastate_main_govt_inv_incomp_cy',
                          'sb_interstate_dyad_count_cy',
                          'sb_interstate_main_govt_inv_incomp_cy', 'ns_dyad_count_cy', 'ns_dyad_ids_cy',
                          'os_dyad_count_cy', 'os_dyad_ids_cy', 'os_main_govt_inv_cy', 'os_any_govt_inv_cy',
                          'os_nsgroup_inv_cy', 'cumulative_total_deaths_parties_in_orgvio_cy',
                          'cumulative_total_deaths_civilians_in_orgvio_cy', 'cumulative_total_deaths_unknown_in_orgvio_cy',
                          'cumulative_total_deaths_in_orgvio_best_cy']]


In [7]:
# Cleaning the main_df

# Importing battle_death data set, to get the dyads 

# importing battle_death dataset
battle_death_df = pd.read_csv("Data/battle_deaths.csv")


# Function to get incompatibility value from battle_death_df based on dyad_id
def get_incompatibility(dyad_ids):
    # Initialize a list to store incompatibility values for each dyad_id
    incompatibilities = []
    
    # Split the dyad_ids separated by ";"
    dyad_ids = dyad_ids.split(";")
    
    # Iterate over each dyad_id
    for dyad_id in dyad_ids:
        # Check if dyad_id is 'NO_DYAD'
        if dyad_id != 'NO_DYAD':
            # Look up the incompatibility value from battle_death_df
            incompatibility = battle_death_df.loc[battle_death_df['dyad_id'] == int(dyad_id), 'incompatibility'].values
            # If the dyad_id is found, append the incompatibility value to the list
            if len(incompatibility) > 0:
                incompatibilities.extend(incompatibility)
    
    # Return the maximum incompatibility value found for the dyad_ids
    return max(incompatibilities) if incompatibilities else None

# Create a new column 'incompatibility' in main_df using the function
main_df['incompatibility'] = main_df['sb_dyad_ids_cy'].apply(get_incompatibility)

main_df.insert(main_df.columns.get_loc('sb_dyad_ids_cy') + 1, 'incompatibility', main_df.pop('incompatibility'))

# Replace NaN values from 'incompatibility' column with 0 (= no state incompatibility)
main_df['incompatibility'] = main_df['incompatibility'].fillna(0)

# Change Incompatibility values to integers
main_df['incompatibility'] = main_df['incompatibility'].astype(int)

# Count each occurrence of incompatibility values in main_df
print(main_df['incompatibility'].value_counts())

incompatibility
0    5161
2     896
1     474
3      13
Name: count, dtype: int64


In [9]:
non_state = pd.read_csv("Data/Nonstate_v23_1.csv")

# Convert 'dyad_id' column in 'non_state' DataFrame to string type
non_state['dyad_id'] = non_state['dyad_id'].astype(str)

# Explode the 'ns_dyad_ids_cy' column to create multiple rows for each id
exploded_df = main_df.explode('ns_dyad_ids_cy')

# Merge with 'non_state' DataFrame on 'dyad_id'
merged_df = exploded_df.merge(non_state, how='left', left_on='ns_dyad_ids_cy', right_on='dyad_id')

# Convert 'org' column to string type
merged_df['org'] = merged_df['org'].astype(str)

# Group by the original index and aggregate the 'org' values
grouped_df = merged_df.groupby(merged_df.index)['org'].apply(lambda x: ';'.join(x)).reset_index()

# Merge the aggregated 'org' values back to 'main_df'
main_df = main_df.merge(grouped_df, how='left', left_index=True, right_on='index')

# Drop unnecessary columns except 'ns_dyad_ids_cy'
main_df.drop(columns=['index'], inplace=True)

# Rename the merged column to 'org'
main_df.rename(columns={'org_x': 'org'}, inplace=True)



In [10]:
# Replace "nan" with 0 in 'org' column
main_df['org'] = main_df['org'].replace('nan', 0)

# Drop columns: sb_instrastate_dyad_count, sb_interstate_dyad_count
main_df = main_df.drop(columns=['sb_intrastate_dyad_count_cy', 'sb_interstate_dyad_count_cy'])


In [11]:
# Count the number of occurences of "0" in sb_dyad_count_cy, in ns_dyad_count_cy and in os_dyad_count_cy 
print(main_df['sb_dyad_count_cy'].value_counts())
print(main_df['ns_dyad_count_cy'].value_counts())
print(main_df['os_dyad_count_cy'].value_counts())

sb_dyad_count_cy
0     5161
1      651
2      305
3      182
4      102
5       48
6       28
7       21
8        8
9        7
10       7
11       7
14       6
12       4
15       3
13       3
16       1
Name: count, dtype: int64
ns_dyad_count_cy
0     5773
1      342
2      144
3       73
4       45
5       34
6       30
7       17
8       16
9       14
11      12
10       7
12       5
15       5
24       4
21       3
13       3
17       3
23       2
14       2
20       2
16       2
19       2
22       1
26       1
18       1
35       1
Name: count, dtype: int64
os_dyad_count_cy
0     5162
1      736
2      317
3      132
4       93
5       44
6       15
8       13
10       6
11       5
9        5
7        4
12       4
16       4
15       1
23       1
24       1
13       1
Name: count, dtype: int64


In [ ]:
# Set the figure size
plt.figure(figsize=(12, 8))

# Iterate over each country
for country_name, country_data in education_df.groupby('Country Name'):
    # Plot the education level over the years
    plt.plot(education_df.columns[4:], country_data.iloc[:, 4:].values.flatten(), label=country_name)

# Set labels and title
plt.title('Education Level Over the Years')
plt.xlabel('Year')
plt.ylabel('Education Level (%)')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Move legend outside the graph
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.grid(True)
plt.tight_layout()
# plt.show()

In [ ]:
# Trying something for the missing data
for country_name, country_data in education_df.groupby('Country Name'):
    # Interpolate missing values using linear interpolation
    education_df.loc[education_df['Country Name'] == country_name, '1989':'2023'] = country_data.loc[:, '1989':'2023'].interpolate(method='linear', axis=1)

# Set the figure size
plt.figure(figsize=(12, 8))

# Iterate over each country
for country_name, country_data in education_df.groupby('Country Name'):
    # Plot the education level over the years
    plt.plot(education_df.columns[4:], country_data.iloc[:, 4:].values.flatten(), label=country_name)

# Set labels and title
plt.title('Education Level Over the Years (Filled Data)')
plt.xlabel('Year')
plt.ylabel('Education Level (%)')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Move legend outside the graph
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.grid(True)
plt.tight_layout()
# plt.show()